In [1]:
import numpy as np

import pandas as pd

In [2]:
from zipline.pipeline.factors import CustomFactor
from zipline.pipeline.data import CNEquityPricing

from zipline.pipeline.fundamentals import Fundamentals

from zipline.pipeline import Pipeline
from zipline.research import run_pipeline, select_output_by, symbol

# 自定义因子

In [3]:
def continuous_num(bool_values, include=False):
    """尾部连续为真累计数"""
    assert bool_values.ndim == 2, "数据ndim必须为2"
    assert bool_values.dtype is np.dtype('bool'), '数据类型应为bool'

    def get_count(x):
        count = 0
        if not include:
            # 自尾部循环，一旦存在假，则返回
            for v in reversed(x):
                if not v:
                    break
                count += 1
        else:
            count = np.count_nonzero(x)
        return count

    return pd.DataFrame(bool_values).apply(get_count).values

## `SuccessiveYZ`

In [4]:
class SuccessiveYZ(CustomFactor):
    """
    尾部窗口连续一字数量

    Parameters
    ----------
    include : bool
        是否计算全部一字数量，若为否，则只计算尾部连续一字数量
        可选参数。默认为否。
    include_st : bool
        可选参数。是否包含ST股票。默认包含。
    window_length : int
        可选参数。默认为100天，不小于3。

    returns
    -------
        连续一字涨停数量， 连续一字跌停数量

    Notes:
    ------
        截至当前连续一字板数量。
        1. 最高 == 最低
        2. 涨跌幅超限
    """
    params = {'include_st': True, 'include': False}
    inputs = [CNEquityPricing.high, CNEquityPricing.low, CNEquityPricing.close]
    outputs = ['zt', 'dt']
    window_length = 100

    def _validate(self):
        super(SuccessiveYZ, self)._validate()
        if self.window_length < 3:
            raise ValueError('window_length值必须大于2')

    def compute(self, today, assets, out, high, low, close, include_st,
                include):
        limit = 0.04 if include_st else 0.09
        # shape = (window_length - 1, N_assets)
        pct = (close[1:] - close[:-1]) / close[:-1]  # 当日涨跌幅，而非期间涨跌幅
        is_yz = (high == low)[1:]  # 保持形状一致
        is_zt = pct > limit
        is_dt = pct < -limit
        yz_zt = is_zt & is_yz
        yz_dt = is_dt & is_yz
        out.zt = continuous_num(yz_zt, include)
        out.dt = continuous_num(yz_dt, include)

### `300104`

In [5]:
def make_pipeline():
    yz = SuccessiveYZ(window_length=50)
    return Pipeline(columns={
        'close': CNEquityPricing.close.latest,
        '连续一字板跌停数量':yz.dt,
    })

In [6]:
yz_output = run_pipeline(make_pipeline(), '2018-01-24', '2018-02-10')

In [7]:
yz_output.xs(symbol('300104'),level=1)

,close,连续一字板跌停数量
2018-01-24 00:00:00+00:00,30.68,0.0
2018-01-25 00:00:00+00:00,13.80,1.0
2018-01-26 00:00:00+00:00,12.42,2.0
2018-01-29 00:00:00+00:00,11.18,3.0
2018-01-30 00:00:00+00:00,10.06,4.0
2018-01-31 00:00:00+00:00,9.05,5.0
2018-02-01 00:00:00+00:00,8.15,6.0
2018-02-02 00:00:00+00:00,7.34,7.0
2018-02-05 00:00:00+00:00,6.61,8.0
2018-02-06 00:00:00+00:00,5.95,9.0


<div class="burk">
复权处理错误?</div><i class="fa fa-lightbulb-o "></i>

## `NDays`

In [8]:
class NDays(CustomFactor):
    """
    上市天数

    Notes
    -----
    当前日期与上市日期之间自然日历天数    
    """

    # 输入shape = (m days, n assets)
    inputs = [Fundamentals.info.上市日期]
    window_length = 1

    def compute(self, today, assets, out, ts):
        baseline = today.tz_localize(None)
        days = [(baseline - pd.Timestamp(x)).days for x in ts[0]]
        out[:] = days

In [9]:
def make_pipeline():
    return Pipeline(columns={
        '上市天数': NDays(),
    })

In [10]:
ndays = run_pipeline(make_pipeline(), '2018-11-16', '2018-12-27')

In [11]:
select_output_by(ndays, assets=['002942','603187'])

上市天数
2018-11-30 00:00:00+00:00 海容冷链(603187)   1.0
2018-12-03 00:00:00+00:00 海容冷链(603187)   4.0
2018-12-04 00:00:00+00:00 海容冷链(603187)   5.0
2018-12-05 00:00:00+00:00 海容冷链(603187)   6.0
2018-12-06 00:00:00+00:00 新农股份(002942)   1.0
                          海容冷链(603187)   7.0
2018-12-07 00:00:00+00:00 新农股份(002942)   2.0
                          海容冷链(603187)   8.0
2018-12-10 00:00:00+00:00 新农股份(002942)   5.0
                          海容冷链(603187)  11.0
2018-12-11 00:00:00+00:00 新农股份(002942)   6.0
                          海容冷链(603187)  12.0
2018-12-12 00:00:00+00:00 新农股份(002942)   7.0
                          海容冷链(603187)  13.0
2018-12-13 00:00:00+00:00 新农股份(002942)   8.0
                          海容冷链(603187)  14.0
2018-12-14 00:00:00+00:00 新农股份(002942)   9.0
                          海容冷链(603187)  15.0
2018-12-17 00:00:00+00:00 新农股份(002942)  12.0
                          海容冷链(603187)  18.0
2018-12-18 00:00:00+00:00 新农股份(002942)  13.0
                          海容冷链(603187)  19.0
2018-12-19 00:00:00+00:00 新农股份(002942)  14.0
                          海容冷链(603187)  20.0
2018-12-20 00:00:00+00:00 新农股份(002942)  15.0
                          海容冷链(603187)  21.0
2018-12-21 00:00:00+00:00 新农股份(002942)  16.0
                          海容冷链(603187)  22.0
2018-12-24 00:00:00+00:00 新农股份(002942)  19.0
                          海容冷链(603187)  25.0
2018-12-25 00:00:00+00:00 新农股份(002942)  20.0
                          海容冷链(603187)  26.0
2018-12-26 00:00:00+00:00 新农股份(002942)  21.0
                          海容冷链(603187)  27.0

## `TradingDays`窗口期有效交易天数

### 查看历史成交量

In [12]:
def make_pipeline():
    return Pipeline(columns={
        'volume': CNEquityPricing.volume.latest,
    })

pipeline_output = run_pipeline(make_pipeline(), '2018-09-01', '2018-10-30')

# 美的集团自2018-09-11停牌至2018-10-29
pipeline_output.xs(symbol('000333'),level=1)

,volume
2018-09-03 00:00:00+00:00,3913.0
2018-09-04 00:00:00+00:00,2229.0
2018-09-05 00:00:00+00:00,2449.0
2018-09-06 00:00:00+00:00,2367.0
2018-09-07 00:00:00+00:00,2636.0
2018-09-10 00:00:00+00:00,2039.0
2018-09-11 00:00:00+00:00,0.0
2018-09-12 00:00:00+00:00,0.0
2018-09-13 00:00:00+00:00,0.0
2018-09-14 00:00:00+00:00,0.0


In [13]:
class TradingDays(CustomFactor):
    """
    窗口期内所有有效交易天数

    Parameters
    ----------
    window_length : 整数
        统计窗口数量

    Notes
    -----
    如成交量大于0,表示当天交易
    """
    inputs = [CNEquityPricing.volume]

    def compute(self, today, assets, out, vs):
        out[:] = np.count_nonzero(vs, 0)

In [14]:
def make_pipeline():
    days = TradingDays(
        window_length=30,
    )    
    return Pipeline(columns={
        'days': days,
    })

In [15]:
pipeline_output = run_pipeline(make_pipeline(), '2018-09-01', '2018-10-30')

In [16]:
select_output_by(pipeline_output, assets=['000333'])

,days
2018-09-03 00:00:00+00:00,30.0
2018-09-04 00:00:00+00:00,30.0
2018-09-05 00:00:00+00:00,30.0
2018-09-06 00:00:00+00:00,30.0
2018-09-07 00:00:00+00:00,30.0
2018-09-10 00:00:00+00:00,30.0
2018-09-11 00:00:00+00:00,29.0
2018-09-12 00:00:00+00:00,28.0
2018-09-13 00:00:00+00:00,27.0
2018-09-14 00:00:00+00:00,26.0


# 过滤器

## `QTradableStocksUS`

In [17]:
def QTradableStocksUS():
    """
    可交易股票(过滤器)

    条件
    ----
        1. 该股票在过去200天内必须有180天的有效收盘价
        2. 并且在最近20天的每一天都正常交易(非停牌状态)
        以上均使用成交量来判定，成交量为0，代表当天停牌
    """
    v20 = TradingDays(window_length=20)
    v200 = TradingDays(window_length=200)
    return (v20 >= 20) & (v200 >= 180)

In [18]:
def make_pipeline():
    return Pipeline(columns={
        'volume': CNEquityPricing.volume.latest,
    })

### 无过滤条件

In [19]:
pipeline_output = run_pipeline(make_pipeline(), '2018-10-30', '2018-10-30')

In [20]:
pipeline_output.shape

(3552, 1)

### 过滤条件

In [21]:
def make_pipeline():
    is_tradeable = QTradableStocksUS()
    return Pipeline(
        columns={
            'volume': CNEquityPricing.volume.latest,
        },
        screen=is_tradeable)

In [22]:
pipeline_output = run_pipeline(make_pipeline(), '2018-10-30', '2018-10-30')
pipeline_output.shape

(2981, 1)

## `IsST`

In [23]:
def IsST():
    """
    当前是否为ST状态

    Notes
    -----
    1. 使用基础数据中股票简称列最新状态值
    2. 如果简称中包含ST字符，则为ST
    """
    short_name = Fundamentals.short_name.latest
    return short_name.has_substring('ST')

In [24]:
def make_pipeline():
    is_st = IsST()
    return Pipeline(
        columns={
            'volume': CNEquityPricing.volume.latest,
            'short_name':Fundamentals.short_name.latest,
        }, screen=is_st)

In [25]:
st_output = run_pipeline(make_pipeline(), '2018-12-27', '2018-12-27')

In [26]:
st_output

volume short_name
2018-12-27 00:00:00+00:00 *ST保千(600074)   782.0      *ST保千
                          ST明科(600091)     68.0       ST明科
                          *ST新亿(600145)     0.0      *ST新亿
                          ST坊展(600149)    138.0       ST坊展
                          *ST船舶(600150)   760.0      *ST船舶
                          *ST创兴(600193)   183.0      *ST创兴
                          *ST大唐(600198)  2146.0      *ST大唐
                          *ST哈空(600202)   109.0      *ST哈空
                          *ST罗顿(600209)   298.0      *ST罗顿
                          ST昌九(600228)    156.0       ST昌九
                          ST山水(600234)     27.0       ST山水
                          *ST椰岛(600238)    90.0      *ST椰岛
                          *ST成城(600247)   143.0      *ST成城

In [27]:
from zipline.pipeline.filters import CustomFilter

## `IsResumed`

In [28]:
class IsResumed(CustomFilter):
    """
    当日是否为停牌后复牌的首个交易日

    Parameters
    ----------
    **Default Inputs:** None

    **Default Window Length:** None

    备注
    ----
        首日成交量为0,次日有成交
        首日市值>0，排除新股上市
    """
    inputs = [CNEquityPricing.volume, CNEquityPricing.close, Fundamentals.equity.流通股本]
    window_safe = True
    window_length = 2

    def _validate(self):
        super(IsResumed, self)._validate()
        if self.window_length != 2:
            raise ValueError('window_length值必须为2')

    def compute(self, today, assets, out, volume, closes, shares):
        is_tp = volume[0] == 0
        is_fp = volume[-1] > 0
        cmv = closes * shares
        not_new = cmv[0] > 0  # 通过流通市值排除新股上市
        out[:] = is_tp & is_fp & not_new

In [29]:
def make_pipeline():
    return Pipeline(
        columns={
            'volume': CNEquityPricing.volume.latest,
            'close': CNEquityPricing.close.latest,
        },
        screen=IsResumed())

In [30]:
fp = run_pipeline(make_pipeline(), '2018-10-01', '2018-10-30')

In [31]:
fp

volume  close
2018-10-08 00:00:00+00:00 鹏翎股份(300375)    284.0   6.68
                          万华化学(600309)   4458.0  42.47
                          新力金融(600318)      9.0   9.94
2018-10-09 00:00:00+00:00 盾安环境(002011)     40.0   5.76
                          爱康科技(002610)   7860.0   2.04
2018-10-10 00:00:00+00:00 国统股份(002205)    572.0  12.33
                          合力泰(002217)    7718.0   5.61
                          乐通股份(002319)    464.0  17.00
                          雷科防务(002413)   1349.0   5.57
                          钢研高纳(300034)    826.0  10.13
                          永清环保(300187)      8.0   8.36
                          千山药机(300216)    648.0   4.46
                          国机汽车(600335)     31.0   9.39
                          城地股份(603887)    139.0  14.85
2018-10-11 00:00:00+00:00 深赤湾Ａ(000022)    297.0  18.72
                          广东甘化(000576)   1899.0   6.38
                          英唐智控(300131)   3009.0   5.31
                          上海临港(600848)    925.0  19.50
2018-10-12 00:00:00+00:00 *ST皇台(000995)   120.0   5.24
                          清新环境(002573)     75.0  10.31
                          宁波富邦(600768)      7.0   9.94
                          新宏泰(603016)      29.0  19.39
2018-10-15 00:00:00+00:00 中 关 村(000931)   805.0   5.81
                          百大集团(600865)    342.0   5.46
2018-10-16 00:00:00+00:00 芒果超媒(300413)    203.0  30.36
2018-10-17 00:00:00+00:00 ST冠福(002102)     84.0   2.07
                          光环新网(300383)    135.0  12.94
                          健康元(600380)     194.0   7.52
                          洲际油气(600759)    133.0   3.55
2018-10-18 00:00:00+00:00 环能科技(300425)    173.0   4.79
                          中国海防(600764)      7.0  28.54
                          丹化科技(600844)     44.0   4.82
2018-10-22 00:00:00+00:00 浩物股份(000757)    194.0   4.46
                          摩登大道(002656)    758.0   8.65
                          天泽信息(300209)    399.0  11.88
                          拓尔思(300229)     879.0   8.15
                          益丰药房(603939)    512.0  47.28
2018-10-23 00:00:00+00:00 京蓝科技(000711)    286.0   9.02
2018-10-25 00:00:00+00:00 加加食品(002650)    492.0   4.37
2018-10-26 00:00:00+00:00 盈峰环境(000967)   1513.0   6.15
                          众信旅游(002707)   1464.0   7.05
2018-10-29 00:00:00+00:00 中粮地产(000031)   2434.0   5.17
                          紫光股份(000938)    835.0  34.42
                          紫光国微(002049)   2080.0  30.72
                          辰安科技(300523)    173.0  55.31
                          广东骏亚(603386)     55.0  21.14
2018-10-30 00:00:00+00:00 国农科技(000004)    198.0  15.49
                          美的集团(000333)   9812.0  36.27
                          小天鹅Ａ(000418)    773.0  42.84
                          金宇车城(000803)     99.0  12.40
                          长信科技(300088)   7710.0   4.89
                          吉药控股(300108)     41.0   7.01
                          汉嘉设计(300746)    445.0  18.18
                          上海沪工(603131)    248.0  13.93

# 组合计算

In [32]:
def market_cap():
    """总市值"""
    return CNEquityPricing.close.latest * Fundamentals.equity.总股本.latest

In [33]:
def make_pipeline():
    return Pipeline(
        columns={
            'volume': CNEquityPricing.volume.latest,
            'close': CNEquityPricing.close.latest,
            'cmv':market_cap()
        },
        screen=IsResumed())

In [34]:
fp = run_pipeline(make_pipeline(), '2018-10-01', '2018-10-30')

In [35]:
fp

volume  close           cmv
2018-10-08 00:00:00+00:00 鹏翎股份(300375)    284.0   6.68  2.403003e+09
                          万华化学(600309)   4458.0  42.47  1.161135e+11
                          新力金融(600318)      9.0   9.94  4.810960e+09
2018-10-09 00:00:00+00:00 盾安环境(002011)     40.0   5.76  5.283142e+09
                          爱康科技(002610)   7860.0   2.04  9.160388e+09
2018-10-10 00:00:00+00:00 国统股份(002205)    572.0  12.33  1.432154e+09
                          合力泰(002217)    7718.0   5.61  1.754982e+10
                          乐通股份(002319)    464.0  17.00  3.400000e+09
                          雷科防务(002413)   1349.0   5.57  6.348389e+09
                          钢研高纳(300034)    826.0  10.13  4.277252e+09
                          永清环保(300187)      8.0   8.36  5.419097e+09
                          千山药机(300216)    648.0   4.46  1.612000e+09
                          国机汽车(600335)     31.0   9.39  9.669229e+09
                          城地股份(603887)    139.0  14.85  2.141370e+09
2018-10-11 00:00:00+00:00 深赤湾Ａ(000022)    297.0  18.72  1.206998e+10
                          广东甘化(000576)   1899.0   6.38  2.825455e+09
                          英唐智控(300131)   3009.0   5.31  5.679185e+09
                          上海临港(600848)    925.0  19.50  2.183843e+10
2018-10-12 00:00:00+00:00 *ST皇台(000995)   120.0   5.24  9.296179e+08
                          清新环境(002573)     75.0  10.31  1.114792e+10
                          宁波富邦(600768)      7.0   9.94  1.329447e+09
                          新宏泰(603016)      29.0  19.39  2.872822e+09
2018-10-15 00:00:00+00:00 中 关 村(000931)   805.0   5.81  4.375668e+09
                          百大集团(600865)    342.0   5.46  2.054272e+09
2018-10-16 00:00:00+00:00 芒果超媒(300413)    203.0  30.36  3.005711e+10
2018-10-17 00:00:00+00:00 ST冠福(002102)     84.0   2.07  5.452041e+09
                          光环新网(300383)    135.0  12.94  1.933995e+10
                          健康元(600380)     194.0   7.52  1.457401e+10
                          洲际油气(600759)    133.0   3.55  8.035452e+09
2018-10-18 00:00:00+00:00 环能科技(300425)    173.0   4.79  3.243170e+09
                          中国海防(600764)      7.0  28.54  1.129520e+10
                          丹化科技(600844)     44.0   4.82  4.899647e+09
2018-10-22 00:00:00+00:00 浩物股份(000757)    194.0   4.46  2.014230e+09
                          摩登大道(002656)    758.0   8.65  6.163297e+09
                          天泽信息(300209)    399.0  11.88  3.453002e+09
                          拓尔思(300229)     879.0   8.15  3.826150e+09
                          益丰药房(603939)    512.0  47.28  1.714820e+10
2018-10-23 00:00:00+00:00 京蓝科技(000711)    286.0   9.02  7.907429e+09
2018-10-25 00:00:00+00:00 加加食品(002650)    492.0   4.37  5.034240e+09
2018-10-26 00:00:00+00:00 盈峰环境(000967)   1513.0   6.15  7.176981e+09
                          众信旅游(002707)   1464.0   7.05  5.986194e+09
2018-10-29 00:00:00+00:00 中粮地产(000031)   2434.0   5.17  9.376992e+09
                          紫光股份(000938)    835.0  34.42  5.022650e+10
                          紫光国微(002049)   2080.0  30.72  1.864145e+10
                          辰安科技(300523)    173.0  55.31  7.964640e+09
                          广东骏亚(603386)     55.0  21.14  4.266052e+09
2018-10-30 00:00:00+00:00 国农科技(000004)    198.0  15.49  1.300799e+09
                          美的集团(000333)   9812.0  36.27  2.403581e+11
                          小天鹅Ａ(000418)    773.0  42.84  2.709578e+10
                          金宇车城(000803)     99.0  12.40  1.583863e+09
                          长信科技(300088)   7710.0   4.89  1.124152e+10
                          吉药控股(300108)     41.0   7.01  4.668763e+09
                          汉嘉设计(300746)    445.0  18.18  3.825072e+09
                          上海沪工(603131)    248.0  13.93  2.786000e+09